In [1]:
import keras.backend as K
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Activation, Dropout, Input
from keras.models import Sequential, Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.optimizers import SGD
from keras.layers.core import Lambda
from keras.engine.topology import Layer, InputSpec

import os
from os import listdir, mkdir
from scipy.misc import imread
import numpy as np
from sklearn.utils import shuffle
from scipy.io import loadmat
import h5py
import sys

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class LocalResponseNormalization(Layer):

    def __init__(self, n=5, alpha=0.0005, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x, mask=None):
        if K.image_dim_ordering == "th":
            _, f, r, c = self.shape
        else:
            _, r, c, f = self.shape
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1, 1),
            padding="same", pool_mode="avg")
        if K.image_dim_ordering == "th":
            summed = K.sum(pooled, axis=1, keepdims=True)
            averaged = self.alpha * K.repeat_elements(summed, f, axis=1)
        else:
            summed = K.sum(pooled, axis=3, keepdims=True)
            averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom

    def get_output_shape_for(self, input_shape):
        return input_shape
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [4]:
from skimage.color import rgb2gray

img_height = 54
img_width = 32
num_channels = 3
num_classes = 2
num_images_train = 208387
num_images_test = 132780+83056

root = '../PKLot/PKLotSegmented/PUC/'

# trainX = np.zeros((num_images_train, img_height, img_width, num_channels))
# validX  = np.zeros((num_images_test, img_height, img_width, num_channels))

# trainY = np.zeros(num_images_train)
# validY = np.zeros(num_images_test)

counter_train = 0
counter_test = 0

sunnyX = np.zeros((208387, img_height, img_width, num_channels))
sunnyY = np.zeros(208387)

if not os.path.isdir('weights'):
    mkdir('weights')

if not os.path.isdir(root):
    print('PKLot dataset folder missing... Exiting!!')
    sys.exit(0)

#------------------------------------------------------------- Preparing Data
print('Preparing data...')

spec_weather = 'Sunny'
root_ = root + spec_weather + '/'
days = [day for day in listdir(root_)]

for day in days:
    _root_ = root_ + day + '/'
    print("Inside folder: %s" % _root_)
    labels = [label for label in listdir(_root_)]
    for i, label in enumerate(labels):
        img_root = _root_ + label + '/'
        img_names = [img for img in listdir(img_root)]
        for j,img_name in enumerate(img_names):
            if (j+1) % 1000 == 0:
                print("Inside folder: %s, image # %d" % (img_root, j+1))
            img = imread(img_root + img_name)
            img = np.resize(img, (img_height, img_width, num_channels))
            sunnyX[counter_train] = img
            sunnyY[counter_train] = 1-i
            counter_train += 1

Preparing data...
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/


/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Empty/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Empty/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Occupied/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Occupied/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Occupied/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Occupied/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Occupied/, image # 6000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Occupied/, image # 7000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-29/Occupied/, image # 8000
Insi

Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-15/Empty/, image # 7000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-15/Empty/, image # 8000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-15/Empty/, image # 9000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-15/Empty/, image # 10000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-15/Empty/, image # 11000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-15/Empty/, image # 12000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-15/Empty/, image # 13000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-15/Empty/, image # 14000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-10-15/Empty/, image # 15000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-18/
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-18/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Sunny/2012-09-18/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC

In [45]:
# rainy_size = 83056
# rainyX = np.zeros((rainy_size, img_height, img_width, num_channels))
# rainyY = np.zeros(rainy_size)
# spec_weather = 'Rainy'
# root_ = root + spec_weather + '/'
# days = [day for day in listdir(root_)]
# counter_train = 0
# counter_test = 0
# for day in days:
#     _root_ = root_ + day + '/'
#     print("Inside folder: %s" % _root_)
#     labels = [label for label in listdir(_root_)]
#     for i, label in enumerate(labels):
#         img_root = _root_ + label + '/'
#         img_names = [img for img in listdir(img_root)]
#         for j,img_name in enumerate(img_names):
#             if (j+1) % 1000 == 0:
#                 print("Inside folder: %s, image # %d" % (img_root, j+1))
#             img = imread(img_root + img_name)
#             img = np.resize(img, (img_height, img_width, num_channels))
#             rainyX[counter_train] = img
#             rainyY[counter_train] = 1-i
#             counter_train += 1

            
cloudy_size = 132780
cloudyX = np.zeros((cloudy_size, img_height, img_width, num_channels))
cloudyY = np.zeros(cloudy_size)
spec_weather = 'Cloudy'
root_ = root + spec_weather + '/'
days = [day for day in listdir(root_)]
counter_train = 0
counter_test = 0
for day in days:
    _root_ = root_ + day + '/'
    print("Inside folder: %s" % _root_)
    labels = [label for label in listdir(_root_)]
    for i, label in enumerate(labels):
        img_root = _root_ + label + '/'
        img_names = [img for img in listdir(img_root)]
        for j,img_name in enumerate(img_names):
            if (j+1) % 1000 == 0:
                print("Inside folder: %s, image # %d" % (img_root, j+1))
            img = imread(img_root + img_name)
            img = np.resize(img, (img_height, img_width, num_channels))
            cloudyX[counter_train] = img
            cloudyY[counter_train] = 1-i
            counter_train += 1      

Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/


/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Empty/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Empty/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Empty/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Occupied/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Occupied/, image # 2000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Occupied/, image # 3000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Occupied/, image # 4000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Occupied/, image # 5000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Occupied/, image # 6000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-31/Occupied/, image # 

Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-14/Empty/, image # 6000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-14/Empty/, image # 7000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-14/Empty/, image # 8000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-14/Empty/, image # 9000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-14/Empty/, image # 10000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-14/Empty/, image # 11000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-14/Empty/, image # 12000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-14/Empty/, image # 13000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-10-14/Empty/, image # 14000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-11-11/
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-11-11/Empty/, image # 1000
Inside folder: ../PKLot/PKLotSegmented/PUC/Cloudy/2012-11-11/Empty/, image # 2000
Inside folder: ../PKLot/PKLotSe

In [8]:
def VGG_16():
    '''Model definition'''

    model = Sequential()
    model.add(Conv2D(64, (11, 11,), padding='valid', strides=(4,4), input_shape=(img_height,img_width,num_channels), name='conv1'))
    model.add(Activation('relu', name='relu1'))
    model.add(LocalResponseNormalization(name='norm1'))
    model.add(MaxPooling2D((2,2), padding='same', name='pool1'))

    model.add(Conv2D(256, (5,5), padding='same', name='conv2'))
    model.add(Activation('relu', name='relu2'))
    model.add(LocalResponseNormalization(name='norm2'))
    model.add(MaxPooling2D((2,2), padding='same', name='pool2'))

    model.add(Conv2D(256, (3, 3), padding='same', name='conv3'))
    model.add(Activation('relu', name='relu3'))
    model.add(Conv2D(256, (3, 3), padding='same', name='conv4'))
    model.add(Activation('relu', name='relu4'))
    model.add(Conv2D(256, (3, 3), padding='same', name='conv5'))
    model.add(Activation('relu', name='relu5'))
    model.add(MaxPooling2D((2,2), padding='same', name='pool5'))

    return model

In [9]:
def copy_mat_to_keras(kmodel, weights_path):
    '''Utility function
    Loads weights from .mat file to keras model
    '''

    kerasnames = [lr.name for lr in kmodel.layers]
    data = loadmat(weights_path, matlab_compatible=False, struct_as_record=False)
    layers = data['layers']
    description = data['meta'][0,0].classes[0,0].description

    prmt = (0,1,2,3)

    for i in range(layers.shape[1]):
        matname = layers[0,i][0,0].name[0]
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            if len(layers[0,i][0,0].weights) > 0:
                l_weights = layers[0,i][0,0].weights[0,0]
                l_bias = layers[0,i][0,0].weights[0,1]
                f_l_weights = l_weights.transpose(prmt)
                assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
                assert (l_bias.shape[1] == 1)
                assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
                assert (len(kmodel.layers[kindex].get_weights()) == 2)
                kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])

In [10]:
model = VGG_16()
# Load pre-trained weights from .mat file
#copy_mat_to_keras(model, 'weights/imagenet-vgg-f.mat')
copy_mat_to_keras(model,'imagenet-vgg-f.mat')
# Freeze the convolutional layers
for layer in model.layers:
    layer.trainable = False

model.add(Flatten())
model.add(Dropout(0.5, name='dropout6'))
model.add(Dense(4096, activation='relu', name='fc6'))
model.add(Dropout(0.5, name='dropout7'))
model.add(Dense(4096, activation='relu', name='fc7'))
model.add(Dropout(0.5, name='dropout8'))
model.add(Dense(1, activation='sigmoid', name='predictions'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 11, 6, 64)         23296     
_________________________________________________________________
relu1 (Activation)           (None, 11, 6, 64)         0         
_________________________________________________________________
norm1 (LocalResponseNormaliz (None, 11, 6, 64)         0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 6, 3, 64)          0         
_________________________________________________________________
conv2 (Conv2D)               (None, 6, 3, 256)         409856    
_________________________________________________________________
relu2 (Activation)           (None, 6, 3, 256)         0         
_________________________________________________________________
norm2 (LocalResponseNormaliz (None, 6, 3, 256)         0         
__________

In [35]:
def run_model(trainX, trainY, validX, validY):
    batch_size = 128
    epochs = 10
    learning_rate = 0.00001
    weight_decay = 0.0005 #regularization
    nesterov = True
    momentum = 0.99

    # optimizer
    sgd = SGD(lr=learning_rate, decay=weight_decay, momentum=momentum, nesterov=nesterov)

    # Callbacks
    tb = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)
#     checkpointer = ModelCheckpoint(filepath="./weights/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5", verbose=1)

    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

    model.fit(trainX, trainY,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(validX, validY))


    probs = model.predict(validX, verbose=1)
    

    label = np.where(probs>0.5,1,0)
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
    print(label.shape)
    print(validY.shape)
    acc =  accuracy_score(label,validY)*100
    print('Accuracy:', acc)
    print('Training complete.')
    return acc

In [48]:
# acc = np.zeros((3, 3))


for i in range(3):
    if i==0:
        trainX = sunnyX
        trainY = sunnyY
        for j in range(3):
#             if j==0:
#                 validX = sunnyX
#                 validY = sunnyY
#             elif j==1:
#                 validX = rainyX
#                 validY = rainyY
            if j==2:
                validX = cloudyX
                validY = cloudyY
                acc[i,j] = run_model(trainX, trainY, validX, validY)
    elif i==1:
        trainX = rainyX
        trainY = rainyY
        for j in range(3):
#             if j==0:
#                 validX = sunnyX
#                 validY = sunnyY
#             elif j==1:
#                 validX = rainyX
#                 validY = rainyY
            if j==2:
                validX = cloudyX
                validY = cloudyY
                acc[i,j] = run_model(trainX, trainY, validX, validY)
    elif i==2:
        trainX = cloudyX
        trainY = cloudyY
        for j in range(3):
            if j==0:
                validX = sunnyX
                validY = sunnyY
            elif j==1:
                validX = rainyX
                validY = rainyY
            elif j==2:
                validX = cloudyX
                validY = cloudyY
            acc[i,j] = run_model(trainX, trainY, validX, validY)


Train on 208387 samples, validate on 132780 samples
Epoch 1/10
208387/208387 [==============================] - 64s 309us/step - loss: 0.1338 - acc: 0.9534 - val_loss: 0.0253 - val_acc: 0.9947
Epoch 2/10
208387/208387 [==============================] - 63s 303us/step - loss: 0.1278 - acc: 0.9546 - val_loss: 0.0260 - val_acc: 0.9948
Epoch 3/10
208387/208387 [==============================] - 64s 305us/step - loss: 0.1258 - acc: 0.9558 - val_loss: 0.0260 - val_acc: 0.9950
Epoch 4/10
208387/208387 [==============================] - 63s 304us/step - loss: 0.1254 - acc: 0.9553 - val_loss: 0.0262 - val_acc: 0.9951
Epoch 5/10
208387/208387 [==============================] - 64s 305us/step - loss: 0.1254 - acc: 0.9555 - val_loss: 0.0267 - val_acc: 0.9949
Epoch 6/10
208387/208387 [==============================] - 63s 304us/step - loss: 0.1239 - acc: 0.9564 - val_loss: 0.0266 - val_acc: 0.9950
Epoch 7/10
208387/208387 [==============================] - 64s 305us/step - loss: 0.1242 - acc: 0.956

In [ ]:
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report

probs = model.predict(validX, verbose=1)

label = np.where(probs>0.5,1,0)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print( 'Accuracy:', accuracy_score(label,validY)*100)

In [49]:
print(acc)

[[97.6855562  98.89712965 99.5149872 ]
 [97.09962714 98.91398574 98.97348998]
 [97.90389996 99.28000385 99.55113722]]
